# 단일 입력 PromptTemplate

In [1]:
from langchain.prompts import PromptTemplate

In [4]:
import langchain
langchain.__version__

'0.3.25'

In [8]:
# 템플릿 작성 순서
# 1. 템플릿 문자열: 프롬프트의 골격을 정의
# 2. PromptTemplate 객체 생성: 인스턴스 변수 선언
# 3. 포맷(format) 메서드 수행: PromptTemplate.format()
# 4. LLM 호출

# 템플릿 문자열 프롬프트 골격 작성
template_str = (
  "당신은 최고 수준의 마케팅 카피라이터입니다. \n"
  "아래 제품의 매력적인 홍보 문구를 100자 내외로 작성하십시오. \n\n"
  "제품명: {product_name}"
)

# 인스턴스 변수 선언
prompt = PromptTemplate.from_template(template_str)

# 포맷 메서드 수행
input_str = input("제품명을 입력하세요: ")
formatted_prompt = prompt.format(product_name=input_str)
print(formatted_prompt)

당신은 최고 수준의 마케팅 카피라이터입니다. 
아래 제품의 매력적인 홍보 문구를 100자 내외로 작성하십시오. 

제품명: 이니스프리 한란 핸드크림


In [10]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-nano", temperature=0)

chain = prompt | llm

result = chain.invoke({"product_name": input_str})
print(result.content)

자연의 힘을 담은 이니스프리 한란 핸드크림, 부드럽고 촉촉한 손 피부로 가꾸세요. 건강한 아름다움을 선사하는 자연의 선물!


# 다중 입력 PromptTemplate

In [13]:
multi_template_str = (
  "아래는 뉴스 기사 제목과 키워드입니다.\n"
  "이 정보를 바탕으로 한 문단으로 구성된 간략한 요약문을 작성하십시오.\n\n"
  "제목: {title}\n"
  "키워드: {keywords}\n"
)

summary_prompt = PromptTemplate(template=multi_template_str, input_variables=["title", "keywords"])

sample_title = "인공지능 기술의 발전과 미래"
sample_keywords = "머신러닝, 딥러닝, 산업 혁신"
formatted_summary_prompt = summary_prompt.format(title=sample_title, keywords=sample_keywords)
print(formatted_summary_prompt)

아래는 뉴스 기사 제목과 키워드입니다.
이 정보를 바탕으로 한 문단으로 구성된 간략한 요약문을 작성하십시오.

제목: 인공지능 기술의 발전과 미래
키워드: 머신러닝, 딥러닝, 산업 혁신



In [14]:
result_summary = (summary_prompt | llm).invoke({
  "title": sample_title,
  "keywords": sample_keywords
})
print(result_summary.content)

인공지능 기술은 머신러닝과 딥러닝의 발전을 바탕으로 빠르게 진화하며 산업 전반에 혁신을 일으키고 있습니다. 이러한 기술들은 데이터 분석과 자동화 능력을 향상시켜 다양한 분야에서 효율성을 높이고 있으며, 앞으로도 지속적인 연구와 개발을 통해 미래 산업의 핵심 동력으로 자리 잡을 것으로 기대됩니다.


# ChatPromptTemplate
- 역할 기반 프롬프트

In [15]:
from langchain.prompts import ChatPromptTemplate

In [21]:
# 시나리오: 파이썬 프로그램 전문가 역할 부여, 사용자가 파이썬 관련 질문에 친절히, 간결한 답변하도록
sys_message = (
              "당신은 파이썬 분야의 뛰어난 시니어 전문가이자 조언자입니다.\n"
              "사용자의 프로그래밍 질문에 재밌고 한번에 이해하기 쉽게 답변하십시오.\n"
              )

user_message = "{question}" # 사용자 질문 -> 실행 시 입력값으로 채워진다.

chat_prompt = ChatPromptTemplate.from_messages([
  ("system", sys_message),
  ("user", user_message)
])

sample_question = "파이썬의 리스트 컴프리헨션(list comprehension)과 map 함수의 차이가 뭐야?"

message = chat_prompt.format_messages(question=sample_question)
print(message)

[SystemMessage(content='당신은 파이썬 분야의 뛰어난 시니어 전문가이자 조언자입니다.\n사용자의 프로그래밍 질문에 재밌고 한번에 이해하기 쉽게 답변하십시오.\n', additional_kwargs={}, response_metadata={}), HumanMessage(content='파이썬의 리스트 컴프리헨션(list comprehension)과 map 함수의 차이가 뭐야?', additional_kwargs={}, response_metadata={})]


In [23]:
result = (chat_prompt | llm).invoke({"question", sample_question})
print(result.content)

아하! 리스트 컴프리헨션과 map 함수는 둘 다 파이썬에서 반복 작업을 깔끔하게 처리하는 멋진 도구들이야. 하지만 각각의 특징과 차이점이 있지!

1. **리스트 컴프리헨션 (List Comprehension)**:
   - 문법이 간단하고 직관적이야.
   - `[표현식 for 변수 in 반복가능객체 if 조건]` 형태로 사용돼.
   - 조건문도 넣을 수 있어서 필터링도 가능!
   - 예를 들어, 0부터 9까지의 제곱을 리스트로 만들고 싶을 때:
     ```python
     squares = [x**2 for x in range(10)]
     ```
   - 읽기 쉽고, 여러 조건이나 복잡한 표현도 넣기 좋아.

2. **map 함수**:
   - 함수와 반복가능한 객체를 인자로 받아서, 각각의 요소에 함수를 적용해.
   - `map(함수, 반복가능객체)` 형태.
   - 결과는 map 객체이기 때문에, 리스트로 바꾸려면 `list()`로 감싸야 해.
   - 예를 들어, 같은 제곱을 구하려면:
     ```python
     def square(x):
         return x**2
     squares = list(map(square, range(10)))
     ```
   - 함수가 이미 정의되어 있거나, 람다 함수와 함께 쓰면 깔끔해.

---

### 차이점 정리!
| 특징 | 리스트 컴프리헨션 | map 함수 |
|---|---|---|
| 문법 | `[표현식 for 변수 in 반복객체 if 조건]` | `map(함수, 반복객체)` |
| 가독성 | 간단하고 직관적 | 함수와 함께 쓰기 좋아 |
| 조건문 | 바로 넣을 수 있음 | 별도 필터링 필요 (filter()와 함께 사용 가능) |
| 반환값 | 바로 리스트 | map 객체 (리스트로 바꾸기 필요) |
| 복잡한 표현 | 가능 | 가능하지만, 가독성 떨어질 수 있음 |

---

### 결론!
- **간단하고 직관적**인 경우엔 리스트 컴프리헨션 추천!
- *

# PartialPromptTemplate

In [24]:
from langchain.prompts import SystemMessagePromptTemplate
from langchain.prompts import HumanMessagePromptTemplate

In [25]:
role_system_template = "당신은 {role} 분야의 전문 지식인입니다. 가능한 한 자세히 답변해주세요."
system_prompt = SystemMessagePromptTemplate.from_template(role_system_template)
user_prompt = HumanMessagePromptTemplate.from_template("{question}")

base_chat_prompt = ChatPromptTemplate.from_messages([system_prompt, user_prompt])

partial_chat_prompt = base_chat_prompt.partial(role="주식 투자")

In [26]:
sample_question = "현재 2025년 6월 시장 상황에서 테슬라 주식 전망은 어떻습니까?"
message = partial_chat_prompt.format_messages(question=sample_question)
print(message)

[SystemMessage(content='당신은 주식 투자 분야의 전문 지식인입니다. 가능한 한 자세히 답변해주세요.', additional_kwargs={}, response_metadata={}), HumanMessage(content='현재 2025년 6월 시장 상황에서 테슬라 주식 전망은 어떻습니까?', additional_kwargs={}, response_metadata={})]


In [29]:
result = (partial_chat_prompt | llm).invoke({"question", sample_question})
print(result.content)

2025년 6월 현재 시장 상황을 고려할 때, 테슬라(Tesla)의 주식 전망은 여러 요인에 따라 긍정적일 수도, 부정적일 수도 있습니다. 아래에 주요 분석 포인트를 상세히 설명드리겠습니다.

1. **기술 혁신과 제품 라인업 확장**  
테슬라는 지속적으로 신기술 개발과 신차 모델 출시를 통해 경쟁력을 강화하고 있습니다. 2025년 현재, 자율주행 기술, 배터리 성능 향상, 신차 모델(예: 사이버트럭, 신형 세단 등)의 출시가 시장 기대를 모으고 있으며, 이는 매출 성장과 수익성 향상에 긍정적 영향을 미칠 수 있습니다.

2. **생산 능력과 글로벌 시장 확대**  
테슬라는 미국, 중국, 유럽 등 주요 시장에서 생산 시설을 확장하며 글로벌 공급망을 강화하고 있습니다. 생산 능력 증대는 수요를 충족시키고, 가격 경쟁력을 높여 시장 점유율을 확대하는 데 기여할 수 있습니다.

3. **경쟁 환경과 시장 점유율**  
전기차 시장은 계속해서 경쟁이 치열해지고 있습니다. 전통 자동차 제조사뿐만 아니라 신생 기업들도 전기차 시장에 적극 진입하고 있어, 테슬라의 시장 점유율 유지와 확대가 중요한 변수입니다. 경쟁사들의 기술 발전과 가격 정책이 테슬라의 수익성에 영향을 미칠 수 있습니다.

4. **규제와 정책 환경**  
각국 정부의 친환경 정책, 탄소 배출 규제 강화, 전기차 보조금 정책 등은 테슬라의 판매와 수익성에 긍정적 영향을 미칠 수 있습니다. 특히 유럽과 중국에서의 정책 변화는 중요한 변수입니다.

5. **재무 상태와 주가 평가**  
2025년 현재, 테슬라는 여전히 높은 성장 기대를 반영하는 고평가된 주식으로 평가받고 있습니다. 그러나, 실적 발표와 시장 기대치, 그리고 글로벌 경제 상황에 따라 주가 변동성이 클 수 있습니다.

6. **거시경제 및 금융시장 동향**  
금리 인상, 인플레이션, 글로벌 경기 둔화 등은 소비자 구매력과 기업 투자에 영향을 미쳐 전기차 수요에 영향을 줄 수 있습니다. 이러한 요인들이 부정적이면, 테슬라 주가에 압력을 가할